# Database connection test

# Preparation

In [1]:
# Imports
import sqlite3
import pandas as pd

## Main connection

In [2]:
# Connect to main database
db_path = '../TsakonianDB.sqlite3'
conn = sqlite3.connect(db_path)

## Auxiliary functions

In [3]:
def query(query: str,
          cursor = conn.cursor()):

      cursor.execute(query)
      fetched = cursor.fetchall()

      if len(fetched) > 0:
            temp_df = pd.DataFrame(fetched, columns=[i[0] for i in cursor.description])
            return temp_df
      else:
            print('Query executed successfully. No results to show.')

# Update tables

## Main

In [4]:
# Load main dictionary
main_df_path = '../data/tables/Main.xlsx'
main_df = pd.read_excel(main_df_path)

# Drop columns with 'duplicate' in the name
main_df = main_df.loc[:, ~main_df.columns.str.contains('duplicate')]

# Remove rows that have NaN in both `nowakowski` and `kostakis` columns
main_df = main_df.dropna(subset=['nowakowski', 'kostakis'], how='all')
main_df

,nowakowski,kostakis,greek,english,paradigm,source_id,lemma,ipa,leonidio,voskina,...,sampatiki,livadi,tyros,melana,sapounakaiika,palaiochora,agios_andreas,kastanitsa,sitaina,prastos
0,άβατθε,άβατ̇ε,άκλαυτος,NaN,Ε,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,άγιε,άγιε,εκκλησία,NaN,Α4,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,άγο,άγο,άλογο,NaN,Α0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,άγουστε,άγουστε,αύγουστος,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,άρζα,άζ̌α,"αραία, όχι συχνά",NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,κρόπο,κρόπο,κοπριά,NaN,Θ0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1067,καμάρα,καμάρα,καμάρα,NaN,Θ,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,κοπέα,κοπέα,κοπέλα,NaN,Θ,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1069,προβάτα,προβάτα,προβατίνα,NaN,Θ,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Save to database
main_df.to_sql('dictionary_entry', 
               conn, 
               if_exists='replace', 
               index=False)

1495

## Sources

In [6]:
# Read sources.xlsx
sources_df_path = '../data/tables/Sources.xlsx'
sources_df = pd.read_excel(sources_df_path)
sources_df.head()

,source_id,title,url
0,1,"Καμβύσης, Ιωάννης (2020). Για να κ̇οντούμε τα ...",NaN
1,2,"Δέφνερ, Μιχαήλ (1923). Λεξικόν της Τσακώνικης ...",NaN
2,3,"Warr, John. Tsakoniandialect.info",http://www.tsakoniandialect.info/
3,4,"Μάνου, Ελένη (2015). Το πρώκιου μι Τσακώνικο λ...",NaN
4,5,"Κωστάκης, Θανάσης (1986). Λεξικό της τσακώνικη...",NaN


In [ ]:
# Copy the sources into the database
sources_df.to_sql('dictionary_source', 
                  conn, 
                  if_exists='replace', 
                  index=False,
                  dtype = {'id': 'bigint',
                           'source': 'varchar(100)',
                           'author': 'varchar(100)',
                           'year': 'varchar(10)',
                           'notes': 'varchar(100)'
                  })

In [ ]:
# Copy the full sources table into the database
sources_df.to_sql('dictionary_source', 
                  conn, 
                  if_exists='replace', 
                  index=False,
                  dtype = {'id': 'bigint',
                           'source': 'varchar(100)',
                           'author': 'varchar(100)',
                           'year': 'varchar(10)',
                           'notes': 'varchar(100)'
                  })

# Delete the temporary table if it exists
try:
    query("DROP TABLE sqlitestudio_temp_table;")
except:
    pass

# Add primary keys and foreign keys by recreating the table
# Obtained from SQLiteStudio
recreating_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                            FROM dictionary_source;

DROP TABLE dictionary_source;

CREATE TABLE dictionary_source (
    source_id   INTEGER       PRIMARY KEY AUTOINCREMENT,
    title       VARCHAR (300),
    url         VARCHAR (200)
);

INSERT INTO dictionary_source (
                                    source_id,
                                    title,
                                    url
                                )
                                SELECT source_id,
                                         title,
                                         url
                                    FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()